In [11]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import heapq 
import pickle
%matplotlib inline
    
import plotnine as p9
import matplotlib.pyplot as plt
import dateutil
import datetime 
from dateutil.rrule import DAILY, rrule, MO, TU, WE, TH, FR

from sklearn.linear_model import LinearRegression, Lasso, BayesianRidge
from yellowbrick.regressor import ResidualsPlot
import statsmodels.formula.api as sm


from sklearn import tree
import math

import warnings
warnings.filterwarnings('ignore')

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


import sys
sys.path.append("../")


In [12]:
with open('../data/linhas.pkl', 'rb') as inputfile:  
    linhas = pickle.load(inputfile)

with open('../data/stops_distances_fixed10.pkl', 'rb') as input_file:
    distancias_pontos = pickle.load(input_file)
    
with open('../data/linesedges.pkl', 'rb') as input_file:
    linesedge = pickle.load(input_file)
    
with open('../data/edgeslines.pkl', 'rb') as input_file:
    edgeslines = pickle.load(input_file)
    
with open('../data/stops_distances.pkl', 'rb') as input_file:
    distancias_paradas = pickle.load(input_file)

In [13]:
for line in range(0,1):    
    linktt = pd.DataFrame(columns=['_id', 'aproxlinkstart', 'hour', 'link', 'ltt', 
                               'month', 'travel_id', 'trip_id', 'weekday'])
    
    for iterated_link in range(0, len(linesedge[linhas[line]])):
    
#     for iterated_link in (range(0, 5)):
        with open('../data/linktt/link-' + str(linesedge[linhas[line]][iterated_link]) + '.pkl', 'rb') as inputfile:  
            linktt_temp = pickle.load(inputfile)  
            linktt = pd.concat([linktt_temp, linktt],sort=False)
  #  start_date = datetime.date(2017, 9, 1)       
   # end_date = datetime.date(2017, 9, 28)  
    linktt['aproxlinkstart'] = pd.to_datetime(linktt['aproxlinkstart'], format = '%H:%M:%S')
    linktt['day'] = pd.to_datetime(linktt['aproxlinkstart'])

    linktt = linktt.loc[(linktt['weekday'] != 1) | (linktt['weekday'] != 7)]

In [14]:
link = 3

# for link in range(2, len(linesedge[linhas[line]])-3):

linktt_k_neighbor = linktt.loc[((linktt['link'] == str(linesedge[linhas[line]][link-2]))   |
                      (linktt['link'] == str(linesedge[linhas[line]][link - 1])) |
                      (linktt['link'] == str(linesedge[linhas[line]][link]))   |
                      (linktt['link'] == str(linesedge[linhas[line]][link+1])) |
                      (linktt['link'] == str(linesedge[linhas[line]][link+2])) 
                      )]

In [15]:
q1, q3 = np.percentile(linktt_k_neighbor['ltt'],[25,75])
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
# lower_bound = 10
upper_bound = q3 + (1.5 * iqr)  

frequency = '30min'            
rolling_win = 1

linktt_k_neighbor.index = linktt_k_neighbor['aproxlinkstart']

linktt_k_neighbor = linktt_k_neighbor.loc[(linktt_k_neighbor['ltt'] >= lower_bound) & 
                                          (linktt_k_neighbor['ltt'] <= upper_bound)] 

linktt_k_neighbor['day'] = pd.to_datetime(np.array(linktt_k_neighbor['aproxlinkstart'])).day

df_temp_grouped  = linktt_k_neighbor.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['ltt'].unstack()\
                .rolling(rolling_win).mean().transform(lambda x: x.fillna(method='ffill')).dropna().reset_index()

df_temp_grouped['aproxlinkstart'] = pd.to_datetime(df_temp_grouped['aproxlinkstart'], format = '%H:%M:%S')

df_temp_grouped['aproxlinkstart'].astype('datetime64[M]').astype(int) % 12 + 1

df_temp_grouped['month'] = pd.to_datetime(np.array(df_temp_grouped['aproxlinkstart'])).month
df_temp_grouped['day'] = pd.to_datetime(np.array(df_temp_grouped['aproxlinkstart'])).day
df_temp_grouped['hour'] = pd.to_datetime(np.array(df_temp_grouped['aproxlinkstart'])).hour
df_temp_grouped['minute'] = pd.to_datetime(np.array(df_temp_grouped['aproxlinkstart'])).minute
df_temp_grouped['weekday'] = df_temp_grouped['aproxlinkstart'].dt.strftime('%w')

In [72]:
def regressão_linear_links vizinhos(month, hour, minute)
    df = df_temp_grouped.loc[(df_temp_grouped['month']==month) & (df_temp_grouped['hour']==hour) & (df_temp_grouped['minute']==minute)]

In [73]:
    df=df.reset_index()
    df

link,index,aproxlinkstart,"(3877, 3878)","(3878, 3879)","(3879, 3880)","(3880, 3881)","(3881, 3882)",month,day,hour,minute,weekday
0,0,2017-01-02 08:30:00,84.882808,43.850302,110.100705,75.689786,24.719121,1,2,8,30,1
1,23,2017-01-03 08:30:00,73.417720,23.165477,84.434301,39.256337,29.898410,1,3,8,30,2
2,51,2017-01-04 08:30:00,65.131053,30.196161,94.777212,51.451726,38.042086,1,4,8,30,3
3,79,2017-01-05 08:30:00,67.045932,14.637876,42.415678,18.199322,17.651652,1,5,8,30,4
4,98,2017-01-06 08:30:00,45.424210,36.217417,96.977551,54.290363,22.709287,1,6,8,30,5
5,125,2017-01-09 08:30:00,43.866934,41.213826,64.610210,50.734223,49.019627,1,9,8,30,1
6,150,2017-01-10 08:30:00,65.974403,25.503181,24.428189,38.497817,42.859044,1,10,8,30,2
7,181,2017-01-11 08:30:00,90.620490,25.886635,71.338768,50.245637,95.077883,1,11,8,30,3
8,207,2017-01-12 08:30:00,92.841295,27.639313,87.650625,52.757666,63.917229,1,12,8,30,4
9,231,2017-01-13 08:30:00,73.163850,30.359863,84.799804,62.967369,67.773544,1,13,8,30,5


In [65]:
    x = df.iloc[:, 1:5].values
    y = df.iloc[:, 5].values

    modelo = LinearRegression()
    modelo.fit(x, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [67]:
    y

array([24.71912092, 29.89841   , 38.04208619, 17.6516521 , 22.709287  ,
       49.01962701, 42.85904377, 95.0778825 , 63.91722863, 67.77354413,
       70.24990548, 33.76040793, 13.52194161, 32.16933839, 38.74698018,
       46.27378255, 30.3711896 , 42.33933856, 16.18193677, 23.23351946,
       62.96530045])

In [66]:
    previsoes = modelo.predict(x)
    previsoes

array([69.88638267, 37.67479949, 43.21142979, 22.18606299, 38.70182596,
       40.01556302, 38.59559444, 53.0714039 , 55.17157866, 53.75011717,
       62.10290685, 21.09443744, 23.56861798, 43.96262119, 35.33011251,
       39.87620381, 35.49313496, 37.16524637, 25.1098434 , 31.29709898,
       54.21654164])

In [80]:
    MAPE = []

    for i in range(len(y)):
        MAPE.append(mean_absolute_percentage_error(y[i], previsoes[i]))

In [81]:
    MAPE

[182.7219580348211,
 26.009374714205048,
 13.58848611642521,
 25.68830876267767,
 70.4229021096081,
 18.36828335535196,
 9.947607201944694,
 44.18112551506589,
 13.68277403723849,
 20.691594543376908,
 11.597166675110321,
 37.51723175985664,
 74.29906642668392,
 36.66001039852626,
 8.818410268341053,
 13.825493370446369,
 16.864487106750552,
 12.220531466626507,
 55.17205240538927,
 34.706663935466025,
 13.894571688815477]